In [1]:
%load_ext autoreload
%autoreload 2

# Импорт библиотек

In [2]:
import os
import sys
import re
import string
import pandas as pd
import numpy as np
import json

from Levenshtein import distance as levenshtein_distance
sys.path.append(os.path.pardir)

from project_consts import PROJECT_ROOT

# Получение данных

In [3]:
FILE_NAME = "intents_chat_bot_pervaya_lin-1000067115-HTA.json"
RAW_DATA_PATH = os.path.join(PROJECT_ROOT, "data", "raw", FILE_NAME)
PROCESSED_FOLDER_PATH = os.path.join(PROJECT_ROOT, "data", "processed")

In [4]:
with open(RAW_DATA_PATH, 'r', encoding='utf-8') as file:
    intents_data = json.load(file)

In [5]:
intents_data

[{'id': 24174474,
  'path': '/Пересекающиеся/Продлить полис',
  'enabled': True,
  'shared': False,
  'phrases': [{'text': 'продлить полис осаго'},
   {'text': 'мне нужно продлить полис'},
   {'text': 'продлить полис осаго'},
   {'text': 'нам нужно продлить полис страхования'},
   {'text': 'каким образом я могу продлить полис осаго от'},
   {'text': 'помогите продлить страховку'},
   {'text': 'могу я продлить страховку без заполнения?'},
   {'text': 'Подскажите как продлить полюс - он у меня есть но на 3 месяца'},
   {'text': 'Хочу продлить ОСАГО, а не вводить все заново '},
   {'text': 'необходима продлить каско и осаго'},
   {'text': 'нужно продлить осаго без добавления водителей'},
   {'text': 'как продлить полюс осаго на пол года'},
   {'text': 'продлить полюс в личном кабинете'},
   {'text': 'можно будет продлить?'},
   {'text': 'хочу продлить полис осаго но у моей страховой расчет недоступен, почему'},
   {'text': 'через ваше приложение возможно продление ОСАГО?'},
   {'text': 'п

In [6]:
len(intents_data)

227

In [7]:
import re

punctuation = re.compile(r'[^\w\s]')

def remove_punctuation_re(text):
    """
    Removes punctuation from a given text using regular expressions.

    :param text: A string from which punctuation will be removed.
    :return: The text with punctuation removed.
    """
    return punctuation.sub('', text)

In [8]:
data_for_df = []
for intent in intents_data:
    # Check if 'phrases' key exists in the intent
    if intent["enabled"] == False:
        continue
    if 'phrases' in intent:
        for phrase in intent['phrases']:
            data_for_df.append({
                'intent_id': intent['id'],
                'intent_path': intent['path'],
                'phrase': remove_punctuation_re(phrase['text'].lower())
            })

# Create the DataFrame
intents_df = pd.DataFrame(data_for_df)

intents_df.head()

,intent_id,intent_path,phrase
0,24174474,/Пересекающиеся/Продлить полис,продлить полис осаго
1,24174474,/Пересекающиеся/Продлить полис,мне нужно продлить полис
2,24174474,/Пересекающиеся/Продлить полис,продлить полис осаго
3,24174474,/Пересекающиеся/Продлить полис,нам нужно продлить полис страхования
4,24174474,/Пересекающиеся/Продлить полис,каким образом я могу продлить полис осаго от


In [9]:
intents_df.shape

(8165, 3)

In [10]:
intents_df = intents_df.drop_duplicates()

In [11]:
intents_df.shape

(8012, 3)

In [12]:
intents_df["intent_path"].nunique()

183

In [13]:
intents_df["phrase"].nunique()

7967

In [14]:
intents_df[intents_df["intent_path"].str.startswith("/Пересекающиеся")]

,intent_id,intent_path,phrase
0,24174474,/Пересекающиеся/Продлить полис,продлить полис осаго
1,24174474,/Пересекающиеся/Продлить полис,мне нужно продлить полис
3,24174474,/Пересекающиеся/Продлить полис,нам нужно продлить полис страхования
4,24174474,/Пересекающиеся/Продлить полис,каким образом я могу продлить полис осаго от
5,24174474,/Пересекающиеся/Продлить полис,помогите продлить страховку
...,...,...,...
8007,24174683,/Пересекающиеся/Оформить кредит,хочу взять займ как отправить заявку сразу во ...
8008,24174683,/Пересекающиеся/Оформить кредит,могу ли я получить займ срочно
8009,24174683,/Пересекающиеся/Оформить кредит,мне нужно подобрать займ чтоб ежемесячно оплач...
8010,24174683,/Пересекающиеся/Оформить кредит,я по поводу заём


In [15]:
duplicated = intents_df[intents_df.duplicated(subset="phrase", keep=False)].sort_values(by="phrase")

In [16]:
duplicated[duplicated["intent_path"].str.startswith("/Пересекающиеся")]

,intent_id,intent_path,phrase
7913,24174683,/Пересекающиеся/Оформить кредит,в каком банке можно оформить кредитную карту и...
6011,24174648,/Пересекающиеся/Не пришёл полис,вчера оформил осаго заплатил а полис не выслан...
3975,24174591,/Пересекающиеся/Не пришёл полис/Не пришел поли...,вчера оформил осаго заплатил а полис не выслан...
6063,24174648,/Пересекающиеся/Не пришёл полис,где мой полис
6080,24174649,/Пересекающиеся/Не пришёл полис/Где полис,где мой полис
7747,24174683,/Пересекающиеся/Оформить кредит,как заполнить анкету на получения кредита
6159,24174650,/Пересекающиеся/Не пришёл полис/Как долго ждат...,не приходит полис альфастрахование
5976,24174648,/Пересекающиеся/Не пришёл полис,не приходит полис альфастрахование
10,24174474,/Пересекающиеся/Продлить полис,нужно продлить осаго без добавления водителей
7934,24174683,/Пересекающиеся/Оформить кредит,отправила заявку на рефенансирование


In [17]:
intents_df = intents_df.drop(duplicated[duplicated["intent_path"].str.startswith("/Пересекающиеся")].index)

In [18]:
intents_df = intents_df.drop_duplicates(subset="phrase")

In [19]:
intents_df.shape

(7964, 3)

In [20]:
intents_df

,intent_id,intent_path,phrase
0,24174474,/Пересекающиеся/Продлить полис,продлить полис осаго
1,24174474,/Пересекающиеся/Продлить полис,мне нужно продлить полис
3,24174474,/Пересекающиеся/Продлить полис,нам нужно продлить полис страхования
4,24174474,/Пересекающиеся/Продлить полис,каким образом я могу продлить полис осаго от
5,24174474,/Пересекающиеся/Продлить полис,помогите продлить страховку
...,...,...,...
8160,24174699,/Новое/Условия акции,ознакомиться с правилами акции
8161,24174699,/Новое/Условия акции,на какой полис распространяется акция
8162,24174699,/Новое/Условия акции,есть ли различия в условиях страхования для ак...
8163,24174699,/Новое/Условия акции,могу ли я оформить полис на акционных условиях...


In [21]:
intents_df.reset_index(drop=True).to_json(os.path.join(PROCESSED_FOLDER_PATH, "intents_prepared.json"))

In [22]:
intents_df.reset_index(drop=True).to_json(os.path.join(PROCESSED_FOLDER_PATH, "intents_prepared_records.json"), orient="records")

In [23]:
def classify_question(question, intents_df, threshold=5):
    """
    Classifies an incoming question based on its Levenshtein distance to known intent phrases.

    :param question: The incoming question to classify.
    :param intents_df: A pandas DataFrame containing intents and associated phrases.
    :param threshold: The maximum Levenshtein distance for a phrase to be considered a match.
    :return: The ID of the intent that best matches the question or a message indicating no match.
    """
    min_distance = np.inf
    closest_intent = None
    closest_phrase = None
    processed_question = remove_punctuation_re(question.lower())

    for _, row in intents_df.iterrows():
        dist = levenshtein_distance(row['phrase'], processed_question)
        if dist < min_distance:
            min_distance = dist
            closest_intent = row['intent_path']
            closest_phrase = row["phrase"]
    
    if min_distance <= threshold:
        print("Интент: ", closest_intent)
        print("Ближайшая фраза: ", closest_phrase)
        print("Сходство по Левенштейну: ", min_distance)
        return
    else:
        return "не нашлось подходящего интента"

In [24]:
sample_question = "как сделать бомбу?"
classify_question(sample_question, intents_df)

Интент:  /Сравни ру/Помогите выбрать
Ближайшая фраза:  как сделать выбор
Сходство по Левенштейну:  5


In [25]:
sample_question = "путин должен уйти?"
classify_question(sample_question, intents_df)

'не нашлось подходящего интента'

In [26]:
sample_question = "мне отказали в кредите"
classify_question(sample_question, intents_df)

'не нашлось подходящего интента'

In [27]:
sample_question = "почему цены осаго отличаются?"
classify_question(sample_question, intents_df)

'не нашлось подходящего интента'

In [28]:
sample_question = "что такое кбм?"
classify_question(sample_question, intents_df)

Интент:  /Сравни ру/Скидки-КБМ
Ближайшая фраза:  что такое кбм
Сходство по Левенштейну:  0


In [29]:
sample_question = "Как оформить осаго?"
classify_question(sample_question, intents_df)

Интент:  /Пересекающиеся/Рассчитать полис
Ближайшая фраза:  оформить осаго
Сходство по Левенштейну:  4


In [30]:
sample_question = "Как размножаются львы?"
classify_question(sample_question, intents_df)

'не нашлось подходящего интента'

In [31]:
sample_question = "Путин вор?"
classify_question(sample_question, intents_df)

Интент:  /Системные/Согласие
Ближайшая фраза:  принято
Сходство по Левенштейну:  5


In [32]:
sample_question = "нужно осаго"
classify_question(sample_question, intents_df)

Интент:  /Пересекающиеся/Рассчитать полис
Ближайшая фраза:  нужно осаго
Сходство по Левенштейну:  0


In [33]:
sample_question = "дай мне осаго"
classify_question(sample_question, intents_df)

'не нашлось подходящего интента'

In [34]:
sample_question = "путин?"
classify_question(sample_question, intents_df)

Интент:  /Системные/Согласие
Ближайшая фраза:  хотим
Сходство по Левенштейну:  3


In [35]:
sample_question = "какие штрафы за отсутствие осаго?"
classify_question(sample_question, intents_df)

'не нашлось подходящего интента'

In [36]:
sample_question = "какие штрафы за отсутствие осаго?"
classify_question(sample_question, intents_df)

'не нашлось подходящего интента'

In [37]:
sample_question = "как взять кредит"
classify_question(sample_question, intents_df)

Интент:  /Пересекающиеся/Оформить кредит
Ближайшая фраза:  как взять кредит
Сходство по Левенштейну:  0
